# Get PAM sites in whole-genome coding regions

This notebook uses python package `gffutils` to parse CDS, and `re` to find positions of NGG PAMs.

This serves as the first layer of reference genome predictions


In [1]:
# move up one-level to access backend
%cd ..

/mnt/ceph/users/zzhang/CROTONdb


In [2]:
import backend
# for watermark
import gffutils
import pandas as pd
import numpy as np
import os
import json
from pyfaidx import Fasta
from tqdm import tqdm

In [3]:
backend.configs.DATA_DIR

'/mnt/ceph/users/zzhang/CROTONdb/backend/../frontend/data'

In [4]:
cds_df = backend.get_CDS_df(db_path=backend.configs.GTF_DB_PATH)

reading gtf_db..


100%|██████████| 774993/774993 [00:31<00:00, 24231.73it/s]


In [5]:
cds_df = backend.get_CDS_seq(df=cds_df, genome_path=backend.configs.GENOME_FA_PATH)

100%|██████████| 772964/772964 [00:42<00:00, 18070.53it/s]
/mnt/ceph/users/zzhang/CROTONdb/backend/get_CDSpams.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['seq'] = seqs


In [6]:
pam_df = backend.get_PAM_coords(df=cds_df)

100%|██████████| 772964/772964 [01:03<00:00, 12170.26it/s]


In [7]:
pam_df.head()

,genename,chrom,start,end,strand,seq,pams,rc_pams
0,OR4F5,chr1,65564,65573,+,ATGAAGAAG,[],[]
1,OR4F5,chr1,69036,70008,+,GTAACTGCAGAGGCTATTTCCTGGAATGAATCAACGAGTGAAACGA...,"[69046, 69130, 69176, 69352, 69586, 69615, 696...","[69909, 69881, 69847, 69756, 69734, 69646, 696..."
2,OR4F5,chr1,69090,70008,+,ATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGATTCTCAGGAAC...,"[69130, 69176, 69352, 69586, 69615, 69648, 699...","[69909, 69881, 69847, 69756, 69734, 69646, 696..."
3,OR4F29,chr1,450739,451678,-,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,"[450838, 450889, 450935, 450968, 450990, 45101...","[451466, 451394, 450989, 450975, 450811, 45080..."
4,OR4F16,chr1,685715,686654,-,ATGGATGGAGAGAATCACTCAGTGGTATCTGAGTTTTTGTTTCTGG...,"[685814, 685865, 685911, 685944, 685966, 68598...","[686442, 686370, 685965, 685951, 685787, 68578..."


In [8]:
# to make 60bp input for CROTON, we pad 33bp to the left of PAM, and 27bp to the right
CDSpamsbed_df, skipped_genes = backend.make_CDSpamsbed(pam_df=pam_df, pam_left=33, pam_right=27)

100%|██████████| 20326/20326 [06:03<00:00, 55.98it/s]


processed PAMs in CDS: (5377203, 7)
skipped genes: []


In [9]:
CDSpamsbed_df = backend.get_ref_PAM_seq(df=CDSpamsbed_df, genome_path=backend.configs.GENOME_FA_PATH)

100%|██████████| 5377203/5377203 [04:44<00:00, 18910.55it/s]


In [10]:
CDSpamsbed_df.head()

,start,end,strand,#,genename,num,pamid,ref_seq
71,69013,69073,+,1,OR4F5,1,OR4F5|1,TCCTTCTCCTTCTCTTCTTCAAGGTAACTGCAGAGGCTATTTCCTG...
74,69024,69084,+,1,OR4F5,2,OR4F5|2,CTCTTCTTCAAGGTAACTGCAGAGGCTATTTCCTGGAATGAATCAA...
25,69031,69091,-,1,OR4F5,3,OR4F5|3,TAGAGTTATTCGTTTCACTCGTTGATTCATTCCAGGAAATAGCCTC...
83,69058,69118,+,1,OR4F5,4,OR4F5|4,GGAATGAATCAACGAGTGAAACGAATAACTCTATGGTGACTGAATT...
90,69079,69139,+,1,OR4F5,5,OR4F5|5,CGAATAACTCTATGGTGACTGAATTCATTTTTCTGGGTCTCTCTGA...


In [11]:
backend.subset_CDSpams(CDSpamsbed_df, save_path=backend.configs.CDS_PAM_DIR, chrom_type='number')

X
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22


In [12]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Sun Aug 28 2022

Python implementation: CPython
Python version       : 3.7.9
IPython version      : 7.22.0

numpy   : 1.21.6
json    : 2.0.9
gffutils: 0.11.0
pandas  : 1.3.5
backend : 0.0.1

Watermark: 2.3.1

